# **Train your own model**


In [ ]:
import sys
sys.argv=['']
del sys
#add other arguments as you need
#please refer to the original main.py file
%run main.py --epochs=30 --use-gpu --balance --ratio-int=2 --f1-loss

# **Use our pretrained model**

In [ ]:
import sys
sys.argv=['']
del sys
import pandas as pd
from main_compare import EchoDataGen
import os
import numpy as np
import tensorflow as tf
from keras.models import model_from_json
from keras.models import load_model
from resnet3d import Model

label_csv= r"C:\Users\User\Desktop\echo_RV\clean_data\five_fold\clean_a4c_0310.csv" #use your own dataframe


def get_test_info(label_csv, fd):
    label_df= pd.read_csv(label_csv)
    test_df= label_df.loc[label_df['train or val']==fd]
    test_df.reset_index(drop= True, inplace= True)
    all_echo=[]
    for i in range(len(test_df)):
        lid= test_df.iloc[i]['lid']
        _set= test_df.iloc[i]['train or val']
        label= test_df.iloc[i]['label']
        echo_tuple= lid, label, _set
        all_echo.append(echo_tuple)
    return all_echo


loaded_model = Model(128, 128, 20)
loaded_model.build(input_shape= (32, 20, 128, 128, 3))

> **All five-fold validation models are provided, load 1-5 models with different weight as you need**\
*Load all five of them if performance evaluation or average prediction are to made*

In [ ]:
loaded_model.load_weights(r"./model/A4C_resnet3d_0.h5")

In [ ]:
#prediction metrics
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
def get_metrics(test_gen, loaded_model):
    tf.keras.utils.set_random_seed(123)  # sets seeds for base-python, numpy and tf
    tf.config.experimental.enable_op_determinism()
    test_gen.on_epoch_end()
    metrics_list=[]
    true_pos= []
    false_pos= []
    false_neg= []
    true_neg= []
    ##also find true_pos and false_pos
    #for batch_number, (echo_batch, label) in enumerate(val_gen):
    for batch_number, (echo_batch, label, lids) in enumerate(test_gen):
        metrics= np.zeros((2, 32))
        out, prob= loaded_model(echo_batch)
        metrics[0]= label[:, 0]
        metrics[1]= prob[:, 0]
        metrics_list.append(metrics)

        pos_pred= np.where(metrics[1]> 0.5)[0].tolist()
        right_prediction= list(filter(lambda x:metrics[0][x]==1, pos_pred))
        wrong_prediction= list(filter(lambda x:metrics[0][x]==0, pos_pred)) #label 0 but prob>0.5
        if len(right_prediction)!=0:
            true_pos.append(list(map(lambda x:lids[int(x)], right_prediction)))
        if len(wrong_prediction)!=0:
            false_pos.append(list(map(lambda x:lids[int(x)], wrong_prediction)))
    
        neg_pred= np.where(metrics[1]< 0.5)[0].tolist()
        right_prediction= list(filter(lambda x:metrics[0][x]==0, neg_pred))
        wrong_prediction= list(filter(lambda x:metrics[0][x]==1, neg_pred)) #label 1 but prob<0.5
        if len(right_prediction)!=0:
            true_neg.append(list(map(lambda x:lids[int(x)], right_prediction)))
        if len(wrong_prediction)!=0:
            false_neg.append(list(map(lambda x:lids[int(x)], wrong_prediction)))
        
    test_metrics= np.concatenate(metrics_list, axis=1)
    return test_metrics


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
#performance is stored in a dictionary
def make_pf_value(test_metrics):
    pf_value={}
   
    y= test_metrics[0]
    scores= test_metrics[1]
    
    pf_value['acc']= accuracy_score(y, scores>0.5)
    pf_value['f1']= f1_score(y, scores>0.5)
    pf_value['precision']= precision_score(y, scores>0.5)
    pf_value['recall']= recall_score(y, scores>0.5)
    pf_value['auroc']= roc_auc_score(y, scores)
    pf_value['auprc']= average_precision_score(y, scores>0.5)
    tp = np.sum(np.logical_and(y == 1, scores>0.5))
    tn = np.sum(np.logical_and(y == 0, scores<=0.5))
    fp = np.sum(np.logical_and(y == 0, scores>0.5))
    fn = np.sum(np.logical_and(y == 1, scores<=0.5))
    pf_value['sensitivity']= tp/(tp+fn)
    pf_value['specificity']= tn/(fp+tn)
    pf_value['fpr']= fp/(tp+fp)
    pf_value['fnr']= fn/(tn+fn)
    
    return pf_value


In [ ]:
#apply youden's value
def make_youdens_value(test_metrics):
    youdens_value={}
   
    y= test_metrics[0]
    scores= test_metrics[1]

    fpr, tpr, thresholds = roc_curve(y, scores)
    j = tpr - fpr
    thresholds = thresholds[thresholds < 1]
    ix = np.argmax(j)
    youdens_thr = thresholds[ix]
    
    youdens_value['acc']= accuracy_score(y, (scores> youdens_thr))
    youdens_value['f1']= f1_score(y,  (scores> youdens_thr))
    youdens_value['precision']= precision_score(y,  (scores> youdens_thr))
    youdens_value['recall']= recall_score(y,  (scores> youdens_thr))
    youdens_value['auroc']= roc_auc_score(y,  (scores> youdens_thr))
    youdens_value['auprc']= average_precision_score(y, (scores> youdens_thr))
    tp = np.sum(np.logical_and(y == 1, scores> youdens_thr))
    tn = np.sum(np.logical_and(y == 0, scores<= youdens_thr))
    fp = np.sum(np.logical_and(y == 0, scores> youdens_thr))
    fn = np.sum(np.logical_and(y == 1, scores<= youdens_thr))
    youdens_value['sensitivity']= tp/(tp+fn)
    youdens_value['specificity']= tn/(fp+tn)
    youdens_value['fpr']= fp/(tp+fp)
    youdens_value['fnr']= fn/(tn+fn)
    
    return youdens_value


## Check the result

In [ ]:

def return_pf(fd, loaded_model, metrics_dict, pf_dict, youdens_dict):

    test_gen= EchoDataGen(
            get_test_info(label_csv, fd),
            balance= False,
            ratio_int= None,
            aug_bool= False,
            batch_size= 32,
            rvi= True,
            sax= True,
            #seg= True,
        )
    test_metrics= get_metrics(test_gen, loaded_model)
    metrics_dict[fd]= test_metrics
    pf_dict[fd]= make_pf_value(test_metrics)
    youdens_dict[fd]= make_youdens_value(test_metrics)

metrics_dict={}
pf_dict={}
youdens_dict={}
return_pf(0, loaded_model, metrics_dict, pf_dict, youdens_dict)


## Performance Evaluation
### Average the performance Dict for all 5-folds

In [ ]:
metrics_dict={}
pf_dict={}
youdens_dict={}
return_pf(0, loaded_model_0, metrics_dict, pf_dict, youdens_dict)
return_pf(1, loaded_model_1, metrics_dict, pf_dict, youdens_dict)
return_pf(2, loaded_model_2, metrics_dict, pf_dict, youdens_dict)
return_pf(3, loaded_model_3, metrics_dict, pf_dict, youdens_dict)
return_pf(4, loaded_model_4, metrics_dict, pf_dict, youdens_dict)
avg_pf={}
for k in pf_dict[0].keys():
    avg_pf[k]=0
    for i in range(len(pf_dict)):
        avg_pf[k]+=pf_dict[i][k]
    avg_pf[k]/=5
    
avg_youdens={}
for k in youdens_dict[0].keys():
    avg_youdens[k]=0
    for i in range(len(youdens_dict)):
        avg_youdens[k]+=youdens_dict[i][k]
    avg_youdens[k]/=5

## Draw Confusion Matrices

In [ ]:
y= metrics_dict[0][0]
scores=  metrics_dict[0][1]
ConfusionMatrixDisplay.from_predictions(y, (scores> 0.5), cmap= 'Oranges')
accuracy= accuracy_score(y, scores>0.5)
plt.title("Threshold 0.5: accuracy {}".format(accuracy))
#plt.savefig(r'C:\Users\User\Desktop\fig1.jpg')
plt.show()

In [ ]:
youdens_value={}
test_metrics= metrics_dict[0]
y= test_metrics[0]
scores= test_metrics[1]

fpr, tpr, thresholds = roc_curve(y, scores)
j = tpr - fpr
thresholds = thresholds[thresholds < 1]
ix = np.argmax(j)
youdens_thr = thresholds[ix]
youdens_accuracy= accuracy_score(y, scores> youdens_thr)
ConfusionMatrixDisplay.from_predictions(y, (scores> youdens_thr), cmap= 'Blues')
plt.title("Optimal threshold: accuracy {}".format(youdens_accuracy))

## Draw AUROC curve


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import roc_curve
tprs = []
base_fpr = np.linspace(0, 1, 101)

plt.figure(figsize=(5, 5))
plt.axes().set_aspect('equal', 'datalim')


for i in range(5):
    
    fpr, tpr, _ = roc_curve(metrics_dict[i][0], metrics_dict[i][1])
    
    plt.plot(fpr, tpr, 'b', alpha=0.15)
    tpr = np.interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)

tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std

import scipy.stats as st

dev= st.tstd(np.array(pf_list_dict['auroc']))
    

plt.plot(base_fpr, mean_tprs, color= 'b', label= f"Average AUROC={round((avg_pf['auroc']), 3)} 95%CI=[{round((avg_pf['auroc']-1.96*dev), 3)},{round((avg_pf['auroc']+1.96*dev), 3)}]")


plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
plt.legend(loc= 'lower right', fontsize= 'small')
plt.plot([0, 1], [0, 1],'darkorange', linestyle= 'dashed')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig(r"./A4C_AUROC.jpg", dpi=600)
plt.show()